## Runnable Interface 介绍与使用

为了尽可能简化创建自定义链的过程，Langchain 实现了一个 **[Runnable](https://api.python.langchain.com/en/stable/runnables/langchain_core.runnables.base.Runnable.html#langchain_core.runnables.base.Runnable)** 协议。

许多 LangChain 组件都实现了 Runnable 协议，包括 chat models, LLMs, output parsers, retrievers, prompt templates等等。此外，还有几个用于处理可运行对象的[有用原语](https://python.langchain.com/v0.1/docs/expression_language/primitives/)。

Runnable 是一个标准接口，包括：

- stream：流式返回生成内容（chunk）
- invoke：对输入调用该链
- batch：对输入列表调用该链

不同组件的输入和输出类型有所差异:

| 组件    | 输入类型                                           | 输出类型           |
| ------------ | ----------------------------------------------------- | --------------------- |
| Prompt       | Dictionary                                            | PromptValue           |
| ChatModel    | Single string, list of chat messages or a PromptValue | ChatMessage           |
| LLM          | Single string, list of chat messages or a PromptValue | String                |
| OutputParser | The output of an LLM or ChatModel                     | Depends on the parser |
| Retriever    | Single string                                         | List of Documents     |
| Tool         | Single string or dictionary, depending on the tool    | Depends on the tool   |


所有 Runnable 对象都显式描述输入和输出 Schema，以检查输入和输出格式：

- input_schema：从Runnable的结构自动生成的输入Pydantic模型
- output_schema：从Runnable的结构自动生成的输出Pydantic模型 


### Input Schema

为了演示如何使用，下面我们创建一个超级简单的PromptTemplate + ChatModel链。

In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

model = ChatOpenAI(model="gpt-4o-mini")
prompt = ChatPromptTemplate.from_template("讲个关于 {topic} 的笑话吧")
chain = prompt | model

#### schema 方法

一个描述 Runnable 接受的输入的说明。这是根据任何 Runnable 结构动态生成的 Pydantic 模型。您可以调用 .schema() 来获取 `JSONSchema` 表示。

In [2]:
# 查看 Chain 的输入类型
chain.input_schema.schema()

{'properties': {'topic': {'title': 'Topic', 'type': 'string'}},
 'required': ['topic'],
 'title': 'PromptInput',
 'type': 'object'}

In [3]:
# 查看 Prompt 的输入类型（Chain的输入从 Prompt 开始，因此输入类型一致）
prompt.input_schema.schema()

{'properties': {'topic': {'title': 'Topic', 'type': 'string'}},
 'required': ['topic'],
 'title': 'PromptInput',
 'type': 'object'}

In [4]:
# 查看 Model 的输入类型
model.input_schema.schema()

{'$defs': {'AIMessage': {'additionalProperties': True,
   'description': 'Message from an AI.\n\nAIMessage is returned from a chat model as a response to a prompt.\n\nThis message represents the output of the model and consists of both\nthe raw output as returned by the model together standardized fields\n(e.g., tool calls, usage metadata) added by the LangChain framework.',
   'properties': {'content': {'anyOf': [{'type': 'string'},
      {'items': {'anyOf': [{'type': 'string'}, {'type': 'object'}]},
       'type': 'array'}],
     'title': 'Content'},
    'additional_kwargs': {'title': 'Additional Kwargs', 'type': 'object'},
    'response_metadata': {'title': 'Response Metadata', 'type': 'object'},
    'type': {'const': 'ai',
     'default': 'ai',
     'enum': ['ai'],
     'title': 'Type',
     'type': 'string'},
    'name': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
     'default': None,
     'title': 'Name'},
    'id': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
     'd

### Output Schema

输出类型仍然可以调用 .schema() 来获取其 `JSONSchema` 表示。

In [5]:
# 查看 Chain 的输出类型
chain.output_schema.schema()

{'$defs': {'AIMessage': {'additionalProperties': True,
   'description': 'Message from an AI.\n\nAIMessage is returned from a chat model as a response to a prompt.\n\nThis message represents the output of the model and consists of both\nthe raw output as returned by the model together standardized fields\n(e.g., tool calls, usage metadata) added by the LangChain framework.',
   'properties': {'content': {'anyOf': [{'type': 'string'},
      {'items': {'anyOf': [{'type': 'string'}, {'type': 'object'}]},
       'type': 'array'}],
     'title': 'Content'},
    'additional_kwargs': {'title': 'Additional Kwargs', 'type': 'object'},
    'response_metadata': {'title': 'Response Metadata', 'type': 'object'},
    'type': {'const': 'ai',
     'default': 'ai',
     'enum': ['ai'],
     'title': 'Type',
     'type': 'string'},
    'name': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
     'default': None,
     'title': 'Name'},
    'id': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
     'd

### Stream

使用 .stream() 方法查看（同步）流式输出结果

In [6]:
for s in chain.stream({"topic": "程序员"}):
    print(s.content, end="", flush=True)

当然可以！这是一个关于程序员的笑话：

有一天，一个程序员走进了咖啡店，看到菜单上有很多饮品。他对咖啡师说：“给我来一杯‘Java’。”

咖啡师笑着问：“你要热的还是冷的？”

程序员回答：“我只要‘热的’版本，不需要‘冷的’因为我已经调试过了！”

希望你喜欢这个笑话！

### Invoke
使用 .invoke() 方法单次（同步）调用

In [7]:
chain.invoke({"topic": "程序员"})

AIMessage(content='当然可以！这是一个关于程序员的笑话：\n\n有一天，一位程序员走进酒吧，点了一杯啤酒。酒保问他：“你要冰啤酒还是常温的？”程序员回答：“我只要代码，没有任何的bug！”\n\n酒保一愣，问：“那你要什么样的啤酒？”程序员微笑着说：“只要能运行就行！”\n\n希望你喜欢这个笑话！', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 98, 'prompt_tokens': 18, 'total_tokens': 116, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_483d39d857', 'finish_reason': 'stop', 'logprobs': None}, id='run-d12b1155-1db1-446f-9dd4-4c3290fab6a1-0', usage_metadata={'input_tokens': 18, 'output_tokens': 98, 'total_tokens': 116})

### Batch
使用 .batch() 方法（同步）批量调用

In [8]:
chain.batch([{"topic": "程序员"}, {"topic": "产品经理"}, {"topic": "测试经理"}])

[AIMessage(content='当然可以！这是一个关于程序员的笑话：\n\n有一天，一个程序员走进咖啡店，点了一杯咖啡。咖啡师问他：“要不要加糖？”\n\n程序员回答：“你先给我一杯不加糖的，我要先测试一下。”\n\n咖啡师疑惑地问：“测试什么？”\n\n程序员说：“测试一下我能不能忍受没有糖的咖啡，如果可以，那我再考虑加糖！”\n\n希望你喜欢这个笑话！', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 108, 'prompt_tokens': 18, 'total_tokens': 126, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_483d39d857', 'finish_reason': 'stop', 'logprobs': None}, id='run-e8f5ac5e-a1c7-40ed-9ffc-6706273d9e83-0', usage_metadata={'input_tokens': 18, 'output_tokens': 108, 'total_tokens': 126}),
 AIMessage(content='当然可以！这里有一个关于产品经理的笑话：\n\n为什么产品经理总是带着铅笔和纸？\n\n因为他们总是准备好“画”出新的想法！', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 16, 'total_tokens': 56, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_483d39d857', 'finish_r

In [10]:
messages = chain.batch([{"topic": "程序员"}, {"topic": "产品经理"}, {"topic": "测试经理"}])

In [11]:
# 使用 StrOutputParser 来处理 Batch 批量输出
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

for idx, m in enumerate(messages):
    print(f"笑话{idx}:\n")
    print(output_parser.invoke(m))
    print("\n")

笑话0:

当然可以！这里有一个关于程序员的笑话：

有一天，一个程序员走进酒吧，点了一杯啤酒。 bartender问他：“你为什么不喝鸡尾酒？”

程序员回答：“因为我不喜欢不确定性。”

希望你喜欢这个笑话！如果想听更多，随时告诉我。


笑话1:

当然可以！这是一个关于产品经理的笑话：

一个产品经理走进一家咖啡店，点了一杯咖啡。他对咖啡师说：“请给我一杯最受欢迎的咖啡！”

咖啡师说：“好的，那我给你一杯拿铁。”

产品经理想了想，又说：“不，我想要一杯用户需求最多的咖啡。”

咖啡师想了想，说：“那我给你一杯白开水吧！”

产品经理：“为什么？”

咖啡师：“因为用户通常只要求‘便宜又好’！”

希望这个笑话能让你开心！


笑话2:

当然可以！这是一个关于测试经理的笑话：

有一天，测试经理和开发经理一起喝咖啡。开发经理自豪地说：“我今天写了一个完美的代码，连一个bug都没有！”

测试经理微微一笑，回答道：“是吗？那你能不能把它发给我，让我来测试一下？”

开发经理满不在乎地说：“没问题！不过我保证你找不到任何问题！”

测试经理拿到代码后，过了一会儿，得意地说：“你看，我就知道你会给我找麻烦！”

开发经理惊讶地问：“为什么这么说？”

测试经理笑着回答：“因为你刚刚说了‘完美’这个词，这已经是一个bug了！” 

希望这个笑话能让你笑一笑！




## 异步操作

这些方法也有相应的异步方法，应与 `asyncio` 的 `await` 语法一起使用以进行并发操作：

- astream：异步地流式返回生成内容（chunk）
- ainvoke：异步地对输入调用该链
- abatch：异步地对输入列表调用该链
- astream_log: 在发生时会返回中间步骤，并且最终返回结果之外。
- astream_events: beta 流式传输事件，在 langchain-core 0.1.14 中引入


### Async Stream

In [12]:
async for s in chain.astream({"topic": "程序员"}):
    print(s.content, end="", flush=True)

当然可以！这里有一个关于程序员的笑话：

有一天，一个程序员走进酒吧，点了一杯啤酒。喝了一口后，他发现啤酒里有个苍蝇。他很生气，叫来 bartender 说：“你看，这杯啤酒里有苍蝇！”

bartender 看了看，微微一笑，回答道：“没关系，喝吧，程序员总是能处理 bug 的！”

程序员苦笑着说：“可我希望能找到一个解决方案，而不是不断调试！”

### Async Invoke

In [13]:
await chain.ainvoke({"topic": "程序员"})

AIMessage(content='当然可以！这是一个关于程序员的笑话：\n\n有一天，一个程序员走进酒吧，点了一杯酒。酒保问他：“你想要什么？”程序员回答：“随便，反正我会把它改成我喜欢的口味！”\n\n酒保无奈地摇了摇头：“你不能修改酒的配方，兄弟！”\n\n程序员微笑着说：“没关系，我可以调试！”\n\n希望你喜欢这个笑话！如果还想听更多，随时告诉我！', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 111, 'prompt_tokens': 18, 'total_tokens': 129, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_54e2f484be', 'finish_reason': 'stop', 'logprobs': None}, id='run-5922b551-e507-4272-860b-4c218d38c6fb-0', usage_metadata={'input_tokens': 18, 'output_tokens': 111, 'total_tokens': 129})

### Async Batch

In [14]:
await chain.abatch([{"topic": "程序员"}, {"topic": "产品经理"}, {"topic": "测试经理"}])

[AIMessage(content='当然可以！这是一个关于程序员的笑话：\n\n有一天，一位程序员走进酒吧，点了一杯啤酒。酒保问他：“你想喝什么类型的啤酒？”程序员回答：“我想要一杯‘无错误’的啤酒！” \n\n酒保愣了一下，随后笑着说：“那你得自己调试一下！” \n\n希望你喜欢这个笑话！', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 92, 'prompt_tokens': 18, 'total_tokens': 110, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_54e2f484be', 'finish_reason': 'stop', 'logprobs': None}, id='run-06efc564-1b4d-466c-998e-3fbea86f4c27-0', usage_metadata={'input_tokens': 18, 'output_tokens': 92, 'total_tokens': 110}),
 AIMessage(content='当然可以！这里有一个关于产品经理的笑话：\n\n有一天，一个产品经理去参加一个派对。他看到一个人正在用一根吸管喝饮料，于是问：“你为什么要用吸管？”\n\n那个人回答：“因为我喜欢这样喝，感觉更方便。”\n\n产品经理想了想，微笑着说：“你知道吗？我们可以把这个吸管做成智能的，能根据饮料的温度自动调节吸入的力度！”\n\n那个人惊讶地说：“哇，那你真是个天才！”\n\n产品经理自信地回答：“不，我只是个产品经理，我只负责想点子，至于实际效果……那是另一个团队的事！”\n\n希望这个笑话能让你开心！', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 162, 'prompt_tokens': 16, 'total_tok